In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ga import Solution, Population

# Genetic Algorithm, applied to game AI

## Genetic Algorithm (GA)

### A Hello World

Dummy problem:
* Find 3 float: f0, f1, f2 (all > 0)
* f0 + f1 + f2 has to be close to 10,000
* f1 > f2
* f0 has to be close to the square value of an integer and greater than the square value

Inspired from natural evolution:
* A solution = set of genes

In [ ]:
class Solution:
    def __init__(self, limit=10000):
        self.f0 = random() * limit
        self.f1 = random() * limit
        self.f2 = random() * limit
        self.generation = 0
        self.score = None

* Evaluate how fit is the solution to the problem

In [ ]:
    def evaluate(self):
        s = self.f0 + self.f1 + self.f2                 # f0 + f1 + f2 has to be close to 10,000
        delta = self.f1 - self.f2                       # f1 > f2
        delta_square = self.f0 - int(sqrt(self.f0))**2  # eg: f0 = 95, int(sqrt(f0))~ int(9.7) = 9, 95-9x9 = 14 
        
        score = - abs(10000 - s) * 10      # has to be close to 0
        score += 1000 if delta > 0 else 0  # bonus if f1 > f2
        score += - delta_square / 1000     # has to be close to 0, but not super important
        
        self.score = score

* Keep the best ones (survival of the fittest)
* Recreate a full population from the best solutions:
    * Clone and mutate a solution
    

In [ ]:
    def clone_and_mutate(self, generation):
        new_solution = Solution()
        new_solution.f0 = self.f0
        new_solution.f1 = self.f1
        new_solution.f2 = self.f2
        
        random_index = randint(0,2)
        if random_index == 0:
            new_solution.f0 = normal(new_solution.f0, new_solution.f0 * 0.2)
        elif random_index == 1:
            new_solution.f1 = random()*10000
        else:
            new_solution.f2 += max(0, random() * 1000 - 500)
        
        new_solution.generation = generation
        
        return new_solution

    * Cross breed 2 solutions

In [ ]:
    def cross(self, another_solution, generation):
        new_solution = Solution()
        new_solution.f0 = self.f0              # f0 from first parent
        new_solution.f1 = another_solution.f1  # f1 and f2 from second parent
        new_solution.f2 = another_solution.f2
        
        new_solution.generation = generation
        
        return new_solution

Test:

In [3]:
s0 = Solution(limit=100)
s0.print()
s0.evaluate()

score: NA, f0: 64.375 f1: 53.532 f2: 44.626, sum=162.533, f1 > f2? True, f0 ~ 8**2 = 64, gen: 0


* Generate a population of solutions for the problem

In [ ]:
class Population:
    def __init__(self, n_solutions, n_keep_best, limit=10000):
        self.n_solutions = n_solutions
        self.n_keep_best = n_keep_best
        
        self.solutions = [Solution(limit=limit) for i in range(self.n_solutions)]
        self.generations = 0        

* Iterate once:

In [ ]:
    def iterate(self):
        self.generations += 1
        
        # EVALUTATE
        for solution in self.solutions:
            solution.evaluate()
            
        # KEEP BEST
        self.solutions.sort(key=lambda x: x.score, reverse=True)
        self.solutions = self.solutions[:self.n_keep_best]

        # print the new best solution if any
        if self.solutions[0].generation == self.generations-1:
            self.solutions[0].print()
        
        # REPRODUCTION
        # one mutant clone of each best solution
        self.solutions.extend([solution.clone_and_mutate(self.generations) for solution in self.solutions])
        
        # crossing, uniform random parents selection
        while len(self.solutions) < self.n_solutions:
            first_parent_index = randint(0, self.n_keep_best-1)
            second_parent_index = randint(0, self.n_keep_best-1)
            while first_parent_index == second_parent_index:
                second_parent_index = randint(0, self.n_keep_best-1)
                
            self.solutions.append(
                self.solutions[first_parent_index].cross(
                    self.solutions[second_parent_index], self.generations)
            )

* Rinse and repeat:

In [ ]:
    def evolution(self, max_generation):
        generation = 0
        while generation < max_generation:
            generation += 1
            self.iterate()

### Examples

* 200 solutions
* keep top 50 after each iteration
* initial fi < 1000
* 100 generations

In [4]:
population = Population(20, 5, limit=10000)
population.evolution(10000)

score: -373.374, f0: 5555.712 f1: 2907.570 f2: 1674.048, sum=10137.329, f1 > f2? True, f0 ~ 74**2 = 5476, gen: 0
score: 155.347, f0: 5244.008 f1: 2907.570 f2: 1763.963, sum=9915.541, f1 > f2? True, f0 ~ 72**2 = 5184, gen: 8
score: 339.504, f0: 5244.008 f1: 2925.985 f2: 1763.963, sum=9933.956, f1 > f2? True, f0 ~ 72**2 = 5184, gen: 13
score: 778.250, f0: 5332.226 f1: 2925.985 f2: 1763.963, sum=10022.175, f1 > f2? True, f0 ~ 73**2 = 5329, gen: 18
score: 957.512, f0: 5314.287 f1: 2925.985 f2: 1763.963, sum=10004.236, f1 > f2? True, f0 ~ 72**2 = 5184, gen: 55
score: 992.312, f0: 5309.295 f1: 2925.985 f2: 1763.963, sum=9999.244, f1 > f2? True, f0 ~ 72**2 = 5184, gen: 92
score: 997.505, f0: 5309.295 f1: 2925.985 f2: 1764.482, sum=9999.763, f1 > f2? True, f0 ~ 72**2 = 5184, gen: 427
score: 999.294, f0: 5309.590 f1: 2925.985 f2: 1764.482, sum=10000.058, f1 > f2? True, f0 ~ 72**2 = 5184, gen: 602
score: 999.826, f0: 5309.537 f1: 2925.985 f2: 1764.482, sum=10000.005, f1 > f2? True, f0 ~ 72**2 = 

In [5]:
population = Population(200, 50, limit=10000)
population.evolution(10000)

score: 336.893, f0: 350.006 f1: 6178.974 f2: 3537.328, sum=10066.308, f1 > f2? True, f0 ~ 18**2 = 324, gen: 0
score: 995.476, f0: 284.148 f1: 6178.974 f2: 3537.328, sum=10000.450, f1 > f2? True, f0 ~ 16**2 = 256, gen: 1
score: 998.382, f0: 283.539 f1: 6178.974 f2: 3537.328, sum=9999.841, f1 > f2? True, f0 ~ 16**2 = 256, gen: 31
score: 999.583, f0: 283.737 f1: 6178.974 f2: 3537.328, sum=10000.039, f1 > f2? True, f0 ~ 16**2 = 256, gen: 49
score: 999.844, f0: 283.711 f1: 6178.974 f2: 3537.328, sum=10000.013, f1 > f2? True, f0 ~ 16**2 = 256, gen: 428
score: 999.875, f0: 283.689 f1: 6178.974 f2: 3537.328, sum=9999.990, f1 > f2? True, f0 ~ 16**2 = 256, gen: 874
score: 999.917, f0: 283.689 f1: 6178.974 f2: 3537.332, sum=9999.994, f1 > f2? True, f0 ~ 16**2 = 256, gen: 929
score: 999.920, f0: 283.689 f1: 6178.974 f2: 3537.332, sum=9999.995, f1 > f2? True, f0 ~ 16**2 = 256, gen: 994
score: 999.948, f0: 283.689 f1: 6178.976 f2: 3537.332, sum=9999.998, f1 > f2? True, f0 ~ 16**2 = 256, gen: 1260
sc

In [6]:
population = Population(1000, 200, limit=10000)
population.evolution(10000)

score: 715.475, f0: 2889.171 f1: 3959.668 f2: 3122.717, sum=9971.556, f1 > f2? True, f0 ~ 53**2 = 2809, gen: 0
score: 921.953, f0: 1614.193 f1: 6208.556 f2: 2169.448, sum=9992.197, f1 > f2? True, f0 ~ 40**2 = 1600, gen: 1
score: 995.226, f0: 3401.285 f1: 5400.072 f2: 1199.117, sum=10000.474, f1 > f2? True, f0 ~ 58**2 = 3364, gen: 2
score: 999.443, f0: 81.829 f1: 7348.705 f2: 2569.411, sum=9999.944, f1 > f2? True, f0 ~ 9**2 = 81, gen: 13
score: 999.760, f0: 81.908 f1: 7348.705 f2: 2569.411, sum=10000.024, f1 > f2? True, f0 ~ 9**2 = 81, gen: 21
score: 999.995, f0: 81.885 f1: 7348.705 f2: 2569.411, sum=10000.000, f1 > f2? True, f0 ~ 9**2 = 81, gen: 35
score: 999.997, f0: 81.884 f1: 7348.705 f2: 2569.411, sum=10000.000, f1 > f2? True, f0 ~ 9**2 = 81, gen: 1091
score: 999.998, f0: 81.884 f1: 7348.705 f2: 2569.411, sum=10000.000, f1 > f2? True, f0 ~ 9**2 = 81, gen: 3463
score: 999.998, f0: 81.884 f1: 7348.705 f2: 2569.411, sum=10000.000, f1 > f2? True, f0 ~ 9**2 = 81, gen: 4243


/!\ Beware of demo effect /!\ , change evaluation function before running.

In [ ]:
population = Population(1000, 200, limit=10000)
population.evolution(10000)

## Game

Code Versus Zombies

Coder Strike Back

## Parameters

* Evaluation function
* Number of generations (exploitation) vs population size (exploration)
* Reproduction strategies:
    * cloning and mutation, mostly when the sequence is important
    * cross breeding:
        * choose parents randomly
        * being a parent depend on score or rank
        * ...
    * ...

## When is it useful

* Easily generate random solutions
* Test and evolve hyper parameters (this why is belongs to *meta heuristics*)
* When a greedy search is too expensive
* Since it's a last resort solution it will certainly be slow
